### Mobile App for Lottery Addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. The [data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018.

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [34]:
#We shall define few useful function here
def factorial(n):
    final_product = 1
    for i in range(n,1,-1):
        final_product *= i
    return final_product

def combinations(n,k):
    return factorial(n)/(factorial(k)*factorial(n-k))

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

In [35]:
def one_ticket_probability(input_list):
    denom = combinations(49, 6)
    prob = (1/denom)*100
    return print("Your winning probability is {0:.7f}%".format(prob))

#Testing our function with a sequence of number
one_ticket_probability([1,12,3,41,25,6])

Your winning probability is 0.0000072%


We'll explore the historical data coming from the Canada 6/49 lottery.

In [37]:
data = pd.read_csv("649.csv")
data.shape
data.head(3)
data.tail(3)

(3665, 11)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


Here we're will write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

In [38]:
#First we shall create a column that contains a set of all 6 numbers drawn
def extract_numbers(row):
    return set(row[4:10])

data['WINNING SET'] = data.apply(extract_numbers,axis=1)
data.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER,WINNING SET
0,649,1,0,6/12/1982,3,11,12,14,41,43,13,"{3, 41, 11, 12, 43, 14}"
1,649,2,0,6/19/1982,8,33,36,37,39,41,9,"{33, 36, 37, 39, 8, 41}"
2,649,3,0,6/26/1982,1,6,23,24,27,39,34,"{1, 6, 39, 23, 24, 27}"
3,649,4,0,7/3/1982,3,9,10,13,20,43,34,"{3, 9, 10, 43, 13, 20}"
4,649,5,0,7/10/1982,5,14,21,31,34,47,45,"{34, 5, 14, 47, 21, 31}"


This function will take a list of 6 numbers, compare with the previous sets of winning numbers and prints number of times the input combination has matched.

We shall test this function will 100 random number drawn from 1 to 49 and see if any combination would have won in the last 3665 times

In [40]:
def check_historical_occurence(input_list, data_col):
    input_set = set(input_list)
    num_of_matches = sum(data_col == input_set)
    return num_of_matches

for i in range(101):
    no_of_wins = 0
    np.random.seed(i)
    my_num = np.random.randint(1,49,6)
    if check_historical_occurence(my_num, data['WINNING SET']) > 0:
        print("You would have won with numbers {} on {} attempt".format(my_num, i))
        no_of_wins += 1
    if i == len(range(101))-1:
        if no_of_wins == 0:
            print("You are unlucky even with {} combinations you chose".format(i))
        

You are unlucky even with 100 combinations you chose


We can see that even after choosing 100 different combinations of random numbers chosen, we would not have one even once in the past.

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We test using the following inputs: [1, 10, 100, 10000, 1000000, 6991908, 13983816].

In [41]:
def multi_ticket_probability(n):
    denom = combinations(49,6)
    winning_prob = (n/denom)*100
    return print("Your chance to win is {0:.7f}%".format(winning_prob), "with {} ticket".format(each))

for each in [1, 10, 100, 10000, 1000000, 6991908, 13983816]:
    multi_ticket_probability(each)

Your chance to win is 0.0000072% with 1 ticket
Your chance to win is 0.0000715% with 10 ticket
Your chance to win is 0.0007151% with 100 ticket
Your chance to win is 0.0715112% with 10000 ticket
Your chance to win is 7.1511238% with 1000000 ticket
Your chance to win is 50.0000000% with 6991908 ticket
Your chance to win is 100.0000000% with 13983816 ticket


Even when playing with 1 million tickets, our chance of winning lotter is just little over 7%.
__________

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

We write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

In [42]:
def probability_less_6(input):
    n_comb_tickets = combinations(6,input)
    n_comb_remaining = combinations(49-input, 6-input)
    num = n_comb_tickets * n_comb_remaining
    denom = combinations(49,6)
    return (num/denom)*100

for input in [2,3,4,5]:
    print("Winning probability is {0:.4f}%".format(probability_less_6(input)))

Winning probability is 19.1327%
Winning probability is 2.1711%
Winning probability is 0.1062%
Winning probability is 0.0019%


With 19% of chance one might get a chance of free play in the next draw and with mere 2% chance one can win $10 at the lottery.